## Lane Detection (for single image)
https://www.hackster.io/kemfic/simple-lane-detection-c3db2f

### Import Libraries

In [2]:
import cv2
import numpy as np
# import matplotlib.pyplot as plt

### Read Images

In [3]:
img_path = r"C:\Users\renac\Downloads\NUS\Y3S1\DSA3101\project\images\2022_01_05_21_10\1701_2103_20220105210501_22826b.jpg" # image at 2110
img = cv2.imread(img_path)

### Display Image

In [4]:
def display_image(img_name, img):
    cv2.namedWindow(img_name, cv2.WINDOW_NORMAL) # fit image to window
    cv2.imshow(img_name, img)
    cv2.waitKey()
    cv2.destroyAllWindows()

### ROI
To mask out region of interest in image
- obtain relevant shape coordinates by clicking on displayed image (min. 4 coordinates required)
- use coordinates to obtain mask image

#### Getting Shape Coordinates
https://www.geeksforgeeks.org/displaying-the-coordinates-of-the-points-clicked-on-the-image-using-python-opencv/

https://stackoverflow.com/questions/23596511/how-to-save-mouse-position-in-variable-using-opencv-and-python

In [5]:
class ShapeCoords:
    def __init__(self):
        self.points = []
    
    def click_event(self, event, x, y, flags, params):
        # checking for left mouse clicks, display in shell if found
        if event == cv2.EVENT_LBUTTONDOWN or event==cv2.EVENT_RBUTTONDOWN:
            self.points.append((x, y))
            # print(x, ' ', y)
            # displaying the coordinates on the image window
            font = cv2.FONT_HERSHEY_SIMPLEX
            text = '(' + str(x) + ', ' + str(y) + ')'
            display_img = img.copy()
            cv2.putText(display_img, text, (x,y), font, 0.8, (255, 0, 0), 2)
            cv2.imshow('image', display_img)

In [6]:
shape_coords = ShapeCoords()

In [7]:
cv2.namedWindow('image', cv2.WINDOW_NORMAL) # fit image to window
cv2.setMouseCallback('image', shape_coords.click_event)
cv2.imshow('image', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [8]:
coords = shape_coords.points
coords

[(0, 616), (634, 294), (876, 199), (1614, 158), (1526, 1077), (0, 1076)]

#### Masking Out ROI
https://www.hackster.io/kemfic/simple-lane-detection-c3db2f

https://pyimagesearch.com/2021/01/19/image-masking-with-opencv/

In [9]:
def roi(img, coords):
    x = int(img.shape[1])
    y = int(img.shape[0])
    if len(coords) < 4:
        print('minimum 4 coordinates required')
        return
    shape = np.array(coords) # shape of roi
    mask = np.zeros_like(img) # np array with zeros (of image dimension)

    # creates a polygon with the mask colour (blue), areas not in roi would be black (pixel is 0)
    cv2.fillPoly(mask, pts=np.int32([shape]), color=(255,255,255))

    # select ares where mask pixels are not zero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [10]:
roi_img = roi(img, coords)
display_image('ROI Image', roi_img)

### Colour Filtering / Masking
To obtain white & yellow lines from roads
- yellow lane lines: get rid of pixels with a hue value outside of 10 and 50 + high Saturation value
- white lane lines: get rid of pixels that have a lightness value < 190

In [11]:
# yellow lane lines: get rid of pixels with a hue value outside of 10 and 50 + high Saturation value
# white lane lines: get rid of pixels that have a lightness value < 190
def filter_white_yellow(img):
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert to rgb

    #convert to HLS to mask based on HLS
    hls_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HLS) # hue, lightness, saturation

    # white mask
    lower = np.array([0,190,0]) # lower bound for white
    upper = np.array([255,255,255]) # upper bound for white
    white_mask = cv2.inRange(hls_img, lower, upper)

    # yellow mask
    yel_lower = np.array([10,0,90]) # lower bound for yellow
    yel_upper = np.array([50,255,255]) # upper bound for yellow
    yellow_mask = cv2.inRange(hls_img, yel_lower, yel_upper)
    
    mask = cv2.bitwise_or(yellow_mask, white_mask)
    masked = cv2.bitwise_and(rgb_img, rgb_img, mask = mask)
    return masked

In [12]:
filter_img = filter_white_yellow(roi_img)
display_image('Filtered Image', filter_img)

### Canny Edge Detection
To obtain edges of lane lines

In [13]:
def canny(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    blur_gray = cv2.GaussianBlur(gray_img, (5, 5), sigmaX=0, sigmaY=0)
    return cv2.Canny(gray_img, 200, 700) # to set upper & lower threshold

In [14]:
canny_img = canny(filter_img)
display_image('Canny Image', canny_img)

#### To Test Canny Threshold
https://stackoverflow.com/questions/25125670/best-value-for-threshold-in-canny

In [ ]:
def callback(x):
    print(x)

canny = cv2.Canny(filter_img, 85, 255) 

cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.createTrackbar('L', 'image', 100, 1000, callback) #lower threshold trackbar for window image
cv2.createTrackbar('U', 'image', 100, 1000, callback) #upper threshold trackbar for window image

while(1):
    # concat_img = cv2.hconcat([filter_img, canny])
    cv2.imshow('image', filter_img)
    k = cv2.waitKey(1) & 0xFF
    if k == 27 or k == 99: # escape key or q
        break
    l = cv2.getTrackbarPos('L', 'image')
    u = cv2.getTrackbarPos('U', 'image')

    canny = cv2.Canny(filter_img, l, u)

cv2.destroyAllWindows()